# Coursework 1 - Crypto
In this course, we work backwards: we start from a known set of working quantitative trading
strategies, we understand how to evaluate them, then how to construct them and intuit them. In the
first step, we solidify our understanding of what we mean by a "working" quant trading strategy". In
this assignment, we examine the basics of evaluating quantitative trading strategies. While they might
sound intimidating at first, in reality, the tools we have to evaluate investor performance are incredibly
powerful relative to have simple they are to use.